In [2]:
import rasterio as rio
import geopandas as gpd
from xgboost import XGBRFRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:


# Sample paths
samples = [
    dict(name="CHM", path="GEDI_GEDI02_A.fgb"),
    dict(name="treecover", path="GEDI_GEDI02_B.fgb"),
    dict(name="AGB", path="GEDI_GEDI04_A.fgb"),
]


In [ ]:
# Function to extract samples from images